<a href="https://colab.research.google.com/github/cpython-projects/da_2603/blob/main/lesson_23_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
DB_USER = "da_test_user"
DB_PASS = "c89M2psBdXGtBXpY3BPrTRZ2bXl6yq2i"
DB_HOST = "dpg-d1hvnobe5dus739ipbcg-a.frankfurt-postgres.render.com"
DB_PORT = "5432"
DB_NAME = "da_test"

In [4]:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

### Общее количество установок

In [5]:
query = text(
    """
    SELECT COUNT(*) FROM app_sessions;
    """
)

df = pd.read_sql(query, engine)
df

,count
0,500


In [6]:
query = text(
    """
    SELECT * FROM app_sessions;
    """
)

df = pd.read_sql(query, engine)
df.shape

(500, 6)

### На Сколько уникальных устройств установили приложение

In [7]:
query = text(
    """
    SELECT COUNT(DISTINCT device_code) FROM app_sessions;
    """
)

df = pd.read_sql(query, engine)
df
#

,count
0,500


### Сколько устройств взаимодействовали с каталогом товаров

In [10]:
query = text(
    """
    SELECT COUNT(DISTINCT device_code) FROM product_views_log;
    """
)

df = pd.read_sql(query, engine)
df

,count
0,500


### Сколько авторизованных пользователей

In [13]:
query = text(
    """
    SELECT COUNT(DISTINCT user_uuid) FROM devices_users_map;
    """
)

df = pd.read_sql(query, engine)
df

,count
0,300


### Сколько пользователей совершили хотя бы одну покупку

In [14]:
query = text(
    """
    SELECT COUNT(DISTINCT user_uuid)
    FROM orders_log
    WHERE total_uah IS NOT NULL AND total_uah > 0;
    """
)

df = pd.read_sql(query, engine)
df

,count
0,300


### Сколько устройств были авторизованы (связаны с user_uuid)

In [15]:
query = text(
    """
    SELECT COUNT(DISTINCT device_code) FROM devices_users_map;
    """
)

df = pd.read_sql(query, engine)
df

,count
0,300


### Конверсия по воронке: установки → авторизация → покупка

In [16]:
query_installs = """
SELECT COUNT(DISTINCT device_code) AS installs FROM app_sessions;
"""

query_authorized = """
SELECT COUNT(DISTINCT device_code) AS authorized FROM devices_users_map;
"""

query_buyers = """
SELECT COUNT(DISTINCT orders_log.user_uuid) AS buyers FROM orders_log;
"""

df_installs = pd.read_sql(query_installs, engine)
df_authorized = pd.read_sql(query_authorized, engine)
df_buyers = pd.read_sql(query_buyers, engine)

In [25]:
installs = df_installs.iloc[0]['installs']
authorized = df_authorized.iloc[0]['authorized']
buyers = df_buyers.iloc[0]['buyers']

print(f"Установка: {installs}")
print(f"Авторизация: {authorized / installs * 100}")
print(f"Покупка: {buyers / authorized * 100}")
print(f'Total: {buyers / installs * 100}')

Установка: 500
Авторизация: 60.0
Покупка: 100.0
Total: 60.0


### Каналы, которые привели к покупкам

### Пользователи, не совершившие ни одной покупки

### Просмотры, но без авторизации